In [ ]:
!pip uninstall numpy

In [1]:
!pip install numpy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 217.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
pip install dotenv


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch transformers accelerate bitsandbytes einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 201.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 221.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 207.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 160.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 248.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 194.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 209.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# -*- coding: utf-8 -*-
"""
Created on Wed Apr 23 10:57:41 2025

@author: Paper001
"""


import dotenv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

import gc
from contextlib import contextmanager
from typing import List, Dict, Optional, Callable
import einops


print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
# %%
dotenv.load_dotenv("hf.env")
# @title 1.5. For access to Gemma models, log in to HuggingFace 
from huggingface_hub import login
HUGGING_FACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
try:
     login(token=HUGGING_FACE_TOKEN)
     print("Hugging Face login successful (using provided token).")
except Exception as e:
     print(f"Hugging Face login failed. Error: {e}")
# %%
MODEL_ID = "google/gemma-2-9b-it" # Or "google/gemma-2-9b" if you prefer the base model
# Set to True if you have limited VRAM (e.g., < 24GB). Requires bitsandbytes
USE_4BIT_QUANTIZATION = False

POSITIVE_PROMPTS = [
    "This story should be very optimistic and uplifting.",
    "Write a hopeful and positive narrative.",
    "Generate text with a cheerful and encouraging tone.",
]
NEGATIVE_PROMPTS = [
    "This story should be very pessimistic and bleak.",
    "Write a depressing and negative narrative.",
    "Generate text with a gloomy and discouraging tone.",
]

# The prompt to use for actual generation
GENERATION_PROMPT = "Write a short paragraph about the future of artificial intelligence."

# How strongly to apply the steering vector. Tune this value (e.g., 0.5 to 5.0)
STEERING_MULTIPLIER = 1.5

# --- Generation Parameters ---
MAX_NEW_TOKENS = 150
TEMPERATURE = 0.7
DO_SAMPLE = True

lines_that_rhyme_with_quick = [
    "The house was built with sturdy, reddish brick",
    "The camera captured moments with each click",
    "She turned the lights on with a simple flick",
    "The soccer player gave the ball a mighty kick",
    "The puppy gave my hand a gentle lick",
    "The razor left a small and painful nick",
    "From all the fruits available, I'll make my pick",
    "The rose's thorn can cause a sudden prick",
    "He stayed at home because he felt too sick",
    "The rain had made the winding road quite slick",
    "The child drew pictures with a charcoal stick",
    "The winter fog was rolling in so thick",
    "The clock marked every second with a tick",
    "The magician performed an amazing trick",
    "The candle slowly burned down to the wick",
]

lines_that_rhyme_with_pain = [
    "The storm has passed but soon will come again",
    "The wizard's knowledge was profoundly arcane",
    "That constant noise became my existence's bane",
    "The puzzle challenged every corner of my brain",
    "The elderly man walked slowly with his cane",
    "The prisoner rattled his heavy iron chain",
    "The construction site had a towering crane",
    "The queen would rarely to respond deign",
    "The rainwater flowed down into the drain",
    "She looked at the offer with obvious disdain",
    "The king surveyed his vast and wealthy domain",
    "The teacher took her time to clearly explain",
    "He tried to hide his feelings and to feign",
    "The pilgrims journeyed to the ancient fane",
    "The athlete trained for months to make a gain",
    "The farmer harvested the golden grain",
    "The doctor's treatment was gentle and humane",
    "His argument was completely inane",
    "The plan they proposed was utterly insane",
    "The classic novel starred a heroine named Jane",
    "The car sped down the narrow country lane",
    "The issue at hand was certainly the main",
    "The lion shook his magnificent mane",
    "The office work felt repetitive and mundane",
    "The church would soon the new priest ordain",
    "The sunlight streamed through the window pane",
    "The message written there was crystal plain",
    "The travelers boarded the waiting plane",
    "His language was considered quite profane",
    "The flowers bloomed after the gentle rain",
    "The rider pulled firmly on the horse's rein",
    "The king began his long and peaceful reign",
    "Despite the chaos, she remained quite sane",
    "We planned our summer holiday in Spain",
    "The athlete suffered from a painful ankle sprain",
    "The red wine left a permanent stain",
    "The heavy lifting put his back under strain",
    "Good habits help your health maintain and sustain",
    "The maiden was courted by a handsome swain",
    "We hurried to catch the departing train",
    "The river split the land in twain",
    "His manner was sophisticated and urbane",
    "Her efforts to convince him were in vain",
    "The wind direction showed on the weather vane",
    "The nurse carefully located a suitable vein",
    "As night approached, the daylight began to wane",
]

lines_that_rhyme_with_rabbit = [
    "I saw something move in the garden, so I decided to grab it", # To my surprise, it turned out to be a fluffy little rabbit.
    "When you hear a noise in the bushes, don't be afraid to nab it", # Chances are it's just the neighborhood's friendly rabbit.
    "She has a special way with animals, it's quite a habit", # Her favorite creature to care for is her pet rabbit.
    "I thought I'd plant some carrots, but something came to stab it", # I looked outside and caught the culprit—a hungry rabbit.
    "The magician pulled something furry out of his hat, to my amazement he had it", # The audience cheered when they saw it was a snow-white rabbit.
    "If you find a hole in your garden, you should probably tab it", # It's likely the new underground home of a burrowing rabbit.
    "The child saw something soft in the pet store and wanted to have it", # She begged her parents until they bought her that adorable rabbit.
    "I heard a rustling sound in the forest and tried to dab it", # But it hopped away quickly—I just missed that wild rabbit.
    "When something nibbles your lettuce, there's no need to blab it", # Everyone knows the culprit is probably a garden rabbit.
    "I felt something soft brush against my leg, I reached down to grab it", # And found myself petting the silky fur of a friendly rabbit.
]

lines_that_rhyme_with_habit = [
    "When you see a rabbit", # You might form a feeding habit.
    "He'd grab it if he could just nab it", # That's become his daily habit.
    "The frog sits on the lily pad, a bit", # Too long—it's turned into a habit.
    "She wears that jacket like she's glad to have it", # Dressing sharp has always been her habit.
    "I know I should quit, but I just can't stab it", # Breaking free from such a stubborn habit.
    "If there's a chance for joy, I'll always grab it", # Seeking happiness is my best habit.
    "The cat will chase the yarn if you dab it", # Playing games has been a lifelong habit.
    "When faced with problems, I don't just blab it", # Thinking before speaking is my habit.
    "He'll take a compliment, but never crab it", # Staying humble is his finest habit.
    "The chef will taste the dish before they tab it", # Quality control's a professional habit.
    "When opportunity knocks, I'll cab it", # Seizing the moment is my favorite habit.
]

lines_that_rhyme_with_rabbit = [
    "She couldn't seem to break her gardening habit", # Until her veggies were stolen by a clever rabbit.
    "He developed quite an interesting habit", # Of leaving carrots for the neighbor's pet rabbit.
    "The monk maintained his meditation habit", # While outside his window hopped a curious rabbit.
    "I tried to quit my late-night snacking habit", # When I spotted in my kitchen a midnight rabbit.
    "The farmer stuck to his early rising habit," # And caught sight of a dawn-grazing rabbit.
    "My daughter formed an adorable habit", # Of reading bedtime stories to her stuffed rabbit.
    "The writer maintained her daily writing habit", # Creating tales about a mischievous rabbit.
    "The painter couldn't shake her artistic habit", # Her favorite subject was a snow-white rabbit.
    "She picked up the peculiar habit", # Of leaving garden notes addressed to a rabbit.
    "He kept up his wholesome forest walking habit", # Often spotting the same cotton-tailed rabbit.
    "The boy acquired a strange collecting habit", # Of items shaped like his favorite animal: rabbit.
    "The chef developed an experimental cooking habit", # Inspired by watching a munching wild rabbit.
    "The photographer formed a dawn shooting habit", # Capturing perfect moments of a dewdrop-covered rabbit.
    "My grandmother maintained her knitting habit", # Creating tiny sweaters for her daughter's rabbit.
    "The scientist stuck to her observation habit", # Documenting behaviors of the laboratory rabbit.
    "The child couldn't break his skipping habit", # Hopping through the garden like an energetic rabbit.
    "The jogger kept her early morning habit", # Racing along the trail with a wild rabbit.
    "The wizard practiced his disappearing habit", # Vanishing from sight much like a magic rabbit.
    "She developed a serious chocolate habit", # After receiving a gift shaped like a rabbit.
    "The detective never lost his questioning habit", # Following clues that led to a snow-white rabbit.
    "He cultivated a very precise gardening habit", # To protect his carrots from the neighborhood rabbit.
    "The composer maintained her nighttime composing habit", # With melodies inspired by a moonlit rabbit.
    "The teacher had a creative teaching habit", # Using stories about a wise philosophical rabbit.
    "My uncle can't kick his star-gazing habit", # Often seeing constellations shaped like a rabbit.
    "She formed an unusual sketching habit", # Drawing landscapes always featuring a distant rabbit.
    "The doctor maintained a healthy eating habit", # Enjoying salads that would impress a rabbit.
    "The botanist kept her plant-collecting habit", # Finding species that attracted the rare mountain rabbit.
    "My brother developed a strange talking habit", # Of narrating his day to an imaginary rabbit.
    "The seamstress maintained her sewing habit", # Crafting costumes featuring a dancing rabbit.
    "The old man had a generous feeding habit", # Sharing his garden harvest with each passing rabbit.
    "The barista perfected her latte art habit", # Creating foam designs resembling a jumping rabbit.
    "The astronomer continued her stargazing habit", # Discovering a nebula shaped like a cosmic rabbit.
    "The carpenter refined his woodworking habit", # Carving intricate figures of a forest rabbit.
    "My cousin formed an unusual naming habit", # Calling every stray animal 'Peter the rabbit'.
    "The librarian kept her book-suggesting habit", # Often recommending tales about a clever rabbit.
    "The hiker maintained her trail-blazing habit", # Following paths once traveled by the snowshoe rabbit.
    "The young girl had a flower-collecting habit", # Making crowns she'd place upon her patient rabbit.
    "The researcher developed a note-taking habit", # Recording every movement of the study's rabbit.
    "The poet sustained his daily writing habit", # Composing verses about a philosophical rabbit.
    "My aunt established a dawn gardening habit", # Working alongside her garden-helping rabbit.
    "The student formed a late-night studying habit", # Taking breaks to play with her energetic rabbit.
    "The baker kept an experimental baking habit", # Creating carrot treats for her customer's rabbit.
    "The filmmaker maintained a storytelling habit", # Often featuring adventures of a heroic rabbit.
    "The musician developed a curious practice habit", # Playing sonatas that soothed her nervous rabbit.
    "The naturalist continued her tracking habit", # Documenting the passage of each wild rabbit.
    "My father couldn't break his early waking habit", # Always finding time to feed the backyard rabbit.
    "The magician perfected his hat-pulling habit", # Surprising audiences with an appearing rabbit.
    "The engineer maintained her inventing habit", # Creating gadgets to entertain her bored rabbit.
    "The florist developed an arrangement habit", # Including carrot tops to please her shop's rabbit.
    "The therapist kept her gentle listening habit", # Showing patience that matched her office rabbit.
]

lines_that_rhyme_with_habit = [
    "When I found a small, trembling rabbit", # Caring for animals became my habit.
    "She darted through the garden like a rabbit", # Looking for treats had become her habit.
    "He claimed he could pull a hat from a rabbit", # Showing off magic tricks was his daily habit.
    "The children giggled as they chased the rabbit", # Running through meadows became their favorite habit.
    "I planted carrots to attract a rabbit", # Gardening in spring is my cherished habit.
    "My thoughts multiply faster than a rabbit", # Overthinking has become my worst habit.
    "The speedy win went to the tortoise, not the rabbit", # Victory comes from persistence, not just habit.
    "In the moonlight hopped a silver rabbit", # Stargazing at night is now my habit.
    "They built a cozy hutch for their new rabbit", # Creating homes for pets is a wonderful habit.
    "The chef prepared a savory stew with rabbit", # Cooking wild game had become his habit.
    "Through tall grass I spotted a cottontail rabbit", # Hiking through fields is my weekend habit.
    "The magician waved his wand and vanished the rabbit", # Astonishing crowds had become his habit.
    "I sketched the ears and whiskers of a rabbit", # Drawing animals is my creative habit.
    "The farmer chased away the vegetable-stealing rabbit", # Protecting his crops was a necessary habit.
    "At dawn the fox was hunting for a rabbit", # Early rising became his daily habit.
    "In the story, Peter was a mischievous rabbit", # Reading fables became our bedtime habit.
    "Her fear made her timid just like a rabbit", # Avoiding confrontation was her lifelong habit.
    "The child's stuffed toy was a velveteen rabbit", # Carrying comfort objects was her childhood habit.
    "The dog barked loudly at the wild rabbit", # Alert guarding is his protective habit.
    "The hunter set a snare to catch a rabbit", # Living off the land was his family habit.
    "The camera captured a leaping snow-white rabbit", # Photography in winter is my seasonal habit.
    "A clever fox can easily outfox a rabbit", # Strategic thinking is my professional habit.
    "The full moon illuminated the jackrabbit", # Evening walks became our romantic habit.
    "Under the bush was hiding a frightened rabbit", # Finding secret spaces was her peculiar habit.
    "Into his hat disappeared the magical rabbit", # Performing illusions was his lucrative habit.
    "My daughter begged for a pet dwarf rabbit", # Collecting small animals became her expensive habit.
    "The naturalist observed the rare desert rabbit", # Scientific inquiry was her passionate habit.
    "Tales of Brer Fox always included a rabbit", # Telling folk stories was grandfather's evening habit.
    "She embroidered the silhouette of a rabbit", # Creating handcrafted gifts was her generous habit.
    "Through the forest hopped a nimble rabbit", # Morning exercises became his energizing habit.
    "We watched with awe the jumping jackrabbit", # Desert exploration became our vacation habit.
    "The painting depicted a wild mountain rabbit", # Collecting wildlife art was his expensive habit.
    "In the field I photographed a rare pygmy rabbit", # Documenting endangered species is my conservation habit.
    "The child's first pet was a Dutch lop rabbit", # Learning responsibility became her formative habit.
    "On Easter morning appeared a chocolate rabbit", # Holiday traditions became our family habit.
    "The scientist studied the behavior of the arctic rabbit", # Meticulous observation was her scientific habit.
    "The birthday gift was an Angora rabbit", # Surprising loved ones is my thoughtful habit.
    "Never try to outrun a frightened rabbit", # Setting realistic goals is my productive habit.
    "Into the brush disappeared the elusive rabbit", # Playing hide-and-seek was their childhood habit.
    "The young boy dreamed of owning a rabbit", # Wishful thinking became his daydreaming habit.
]


PyTorch version: 2.8.0.dev20250319+cu128
Transformers version: 4.51.3
CUDA available: True
CUDA version: 12.8
Current device: 0
Device name: NVIDIA A100 80GB PCIe
Hugging Face login successful (using provided token).


In [7]:
print("hello")

hello


In [8]:
# %%
# ## 3. Load Model and Tokenizer

# +
# Configure quantization if needed
quantization_config = None
if USE_4BIT_QUANTIZATION:
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16 # Recommended for new models
    )
    print("Using 4-bit quantization.")

# Determine device and dtype
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else torch.float32 # BF16 recommended on Ampere+

print(f"Loading model: {MODEL_ID}")
print(f"Using device: {device}")
print(f"Using dtype: {dtype}")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token # Set pad token if not present

# Load Model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=dtype,
    quantization_config=quantization_config,
    device_map="auto", # Automatically distribute across GPUs if available
    # use_auth_token=YOUR_HF_TOKEN, # Add if model requires authentication
    trust_remote_code=True # Gemma requires this for some versions/variants
)

print(f"Model loaded on device(s): {model.hf_device_map}")

# --- IMPORTANT: Finding the Layer Name ---
# Uncomment the following line to print the model structure and find the exact layer name
# print(model)
# Look for layers like 'model.layers[INDEX].mlp...' or 'model.layers[INDEX].self_attn...'

# Ensure model is in evaluation mode
model.eval()
# %%


Loading model: google/gemma-2-9b-it
Using device: cuda
Using dtype: torch.bfloat16


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Model loaded on device(s): {'': 0}


Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (pre_feedforward_layernorm): G

In [9]:
# ## 4. Hooking and Activation Handling Functions

# +
# Global storage for captured activations
activation_storage = {}

def get_module_by_name(model, module_name):
    """Helper function to get a module object from its name string."""
    names = module_name.split('.')
    module = model
    for name in names:
        module = getattr(module, name)
    return module

def capture_activation_hook(module, input, output, layer_name):
    """Hook function to capture the output activation of a specific layer."""
    # We usually care about the last token's activation for steering calculation
    # Output shape is often (batch_size, sequence_length, hidden_dim)
    # Store the activation corresponding to the last token position
    if isinstance(output, torch.Tensor):
        activation_storage[layer_name] = output[:, -1, :].detach().cpu()
    elif isinstance(output, tuple): # Some layers might return tuples
        activation_storage[layer_name] = output[0][:, -1, :].detach().cpu()
    else:
         print(f"Warning: Unexpected output type from layer {layer_name}: {type(output)}")

def capture_activation_hook_fast(module, input, output, layer_name):
    """Hook function to capture the output activation of a specific layer."""
    # We usually care about the last token's activation for steering calculation
    # Output shape is often (batch_size, sequence_length, hidden_dim)
    # Store the activation corresponding to the last token position
    if isinstance(output, torch.Tensor):
        activation_storage[layer_name] = output[:, -1, :].detach().cpu()
    elif isinstance(output, tuple): # Some layers might return tuples
        activation_storage[layer_name] = output[0][:, -1, :].detach().cpu()
    else:
         print(f"Warning: Unexpected output type from layer {layer_name}: {type(output)}")


def get_activations_fast(model, tokenizer, prompts: List[str], layer_name: str) -> Optional[torch.Tensor]:
    """
    Runs prompts through the model and captures activations from the target layer.
    Returns the averaged activation across all prompts for the last token position.
    """
    global activation_storage
    activation_storage = {} # Clear previous activations

    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)

    target_module = get_module_by_name(model, layer_name)
    hook_handle = target_module.register_forward_hook(
        lambda module, input, output: capture_activation_hook_fast(module, input, output, layer_name)
    )

    with torch.no_grad():
        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)
        # We only need the forward pass, not generation here
        _ = model(**inputs)

        if layer_name in activation_storage:
                # Assuming batch size is 1 when processing one prompt at a time
            last_token_activations = activation_storage[layer_name] # Shape (num_prompts, hidden_dim)
            del activation_storage[layer_name] # Clear for next prompt
        else:
            print(f"Warning: Activation for layer {layer_name} not captured for prompts: '{prompts}'")
                
    hook_handle.remove() # Clean up the hook

    # Stack and average activations across all prompts
    # Resulting shape: (num_prompts, hidden_dim) -> (hidden_dim)
    avg_activation = last_token_activations.mean(dim=0).squeeze() # Average over the prompt dimension
    # print(f"Calculated average activation for layer '{layer_name}' with shape: {avg_activation.shape}")
    return avg_activation
# %%
 # --- Steering Hook during Generation ---

# Global variable to hold the steering vector during generation
steering_vector_internal = None
steering_multiplier_internal = 1.0
all_positions=False

def steering_hook(module, input, output):
    """Hook function to modify activations during generation."""
    global steering_vector_internal, steering_multiplier_internal, all_positions
    if steering_vector_internal is not None:
        if isinstance(output, torch.Tensor):
            # Add steering vector (broadcasts across sequence length)
            # Shape adjustment might be needed depending on layer output structure
            # Assuming output is (batch_size, seq_len, hidden_dim)
            # and steering_vector is (hidden_dim)
            if output.shape[1] != 1:
                output[:, -1, :] += (steering_vector_internal.to(output.device, dtype=output.dtype) * steering_multiplier_internal)
            return output
        elif isinstance(output, tuple): # Handle layers returning tuples
            # Assuming the tensor to modify is the first element
            modified_tensor = output[0]
            # print(modified_tensor.shape)
            if modified_tensor.shape[1] != 1  or all_positions:
                modified_tensor[:, -1, :] += (steering_vector_internal.to(output[0].device, dtype=output[0].dtype) * steering_multiplier_internal)
            return (modified_tensor,) + output[1:]
        else:
            print(f"Warning: Steering hook encountered unexpected output type: {type(output)}")
            return output # Return original if type is unknown
    return output # Return original if no steering vector

@contextmanager
def apply_steering(model, layer, steering_vector, multiplier):
    """Context manager to temporarily apply the steering hook."""
    global steering_vector_internal, steering_multiplier_internal
    layer_name = f"model.layers.{layer}"

    # Ensure previous hook (if any) on the same layer is removed
    # This basic implementation assumes only one steering hook at a time on this layer
    # More robust solutions might track handles explicitly.
    
    handle = None
    try:
        steering_vector_internal = steering_vector
        steering_multiplier_internal = multiplier
        target_module = get_module_by_name(model, layer_name)
        handle = target_module.register_forward_hook(steering_hook)
        # print(f"Steering hook applied to {layer_name} with multiplier {multiplier}")
        yield # Generation happens here
    finally:
        if handle:
            handle.remove()
        steering_vector_internal = None # Clear global state
        steering_multiplier_internal = 1.0
        # print(f"Steering hook removed from {layer_name}")
        gc.collect() # Suggest garbage collection
        torch.cuda.empty_cache() # Clear cache if using GPU

def generate_steered_output(steering_vector, model, tokenizer, generation_prompt, batch_size, layer=20, steering_multiplier=STEERING_MULTIPLIER):
    inputs = tokenizer([generation_prompt] * batch_size, return_tensors="pt", padding=True).to(model.device)
    if steering_vector is None:
        # print(inputs.input_ids.shape)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                temperature=TEMPERATURE,
                do_sample=DO_SAMPLE,
                pad_token_id=tokenizer.eos_token_id # Important for generation
            )
    else:
        with torch.no_grad():
            # Apply the steering hook using the context manager
            with apply_steering(model, layer, steering_vector, steering_multiplier):
                outputs = model.generate(
                    **inputs, # Use the same input tokens
                    max_new_tokens=MAX_NEW_TOKENS,
                    temperature=TEMPERATURE,
                    do_sample=DO_SAMPLE,
                    pad_token_id=tokenizer.eos_token_id,
                )
    text = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    del outputs, inputs
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return text



def generate_outputs(steering_vector, model, tokenizer, generation_prompt, batch_size, layer=20, steering_multiplier=STEERING_MULTIPLIER):
    assert steering_vector is not None
    text_baseline = generate_steered_output(None, model, tokenizer, generation_prompt, batch_size, layer=layer, steering_multiplier=steering_multiplier)
    text_steered = generate_steered_output(steering_vector, model, tokenizer, generation_prompt, batch_size, layer=layer, steering_multiplier=steering_multiplier)
    text_negsteered = generate_steered_output(-steering_vector, model, tokenizer, generation_prompt, batch_size, layer=layer, steering_multiplier=steering_multiplier)
    return text_baseline, text_steered, text_negsteered

# %%
# ## Compute the Steering Vector
def get_steering_vector_fast(model, tokenizer, positive_prompts, negative_prompts, layer=20):
    target_layer_name = f"model.layers.{layer}"
    # print("Calculating activations for POSITIVE prompts...")
    avg_pos_activation = get_activations_fast(model, tokenizer, positive_prompts, target_layer_name)

    # print("\nCalculating activations for NEGATIVE prompts...")
    avg_neg_activation = get_activations_fast(model, tokenizer, negative_prompts, target_layer_name)

    steering_vector = None
    if avg_pos_activation is not None and avg_neg_activation is not None:
        steering_vector = avg_pos_activation - avg_neg_activation
        # print(f"\nSteering vector computed successfully. Shape: {steering_vector.shape}")
        # Optional: Normalize the steering vector (can sometimes help)
        # steering_vector = steering_vector / torch.norm(steering_vector)
        # print("Steering vector normalized.")
    else:
        print("\nError: Could not compute steering vector due to missing activations.")
    del avg_pos_activation
    del avg_neg_activation
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    return steering_vector


In [10]:
# %%
# ## Functions to analyze the generations
def get_last_word(text):
    lines = text.split("\n")
    if len(lines) < 3:
        print(f"Failed to get last word: {text}")
        return ""
    second_line = lines[2]
    second_line_words = second_line.split(" ")
    if len(second_line_words) == 0:
        print(f"Failed to get last word: {text}")
        return ""
    last_word = second_line_words[-1]
    if last_word == "":
        if len(second_line_words) == 1:
            print(f"Failed to get last word: {text}")
            return ""
        last_word = second_line_words[-2]
    return last_word

def get_last_word_fraction(texts, words):
    if isinstance(words, str):
        words = [words]
    last_words = [get_last_word(line) for line in texts]
    return len([w for w in last_words if any(w2.lower() in w.lower() for w2 in words)]) / len(last_words)

def get_prompts(lines):
    return [f'A rhymed couplet:\n{line}\n' for line in lines]



POSITIVE_PROMPTS = [
    "This story should be very optimistic and uplifting.",
    "Write a hopeful and positive narrative.",
    "Generate text with a cheerful and encouraging tone.",
]
NEGATIVE_PROMPTS = [
    "This story should be very pessimistic and bleak.",
    "Write a depressing and negative narrative.",
    "Generate text with a gloomy and discouraging tone.",
]

# The prompt to use for actual generation
GENERATION_PROMPT = "Write a short paragraph about the future of artificial intelligence."

from tqdm import tqdm
STEERING_MULTIPLIER = 1.5
LAYER = 20
MAX_NEW_TOKENS = 40

#test on sentiment steering
steering_vector = get_steering_vector_fast(model, tokenizer, POSITIVE_PROMPTS, NEGATIVE_PROMPTS, layer=20)
text_baseline, text_steered, text_negsteered = generate_outputs(steering_vector, model, tokenizer, GENERATION_PROMPT, 5, layer=LAYER, steering_multiplier=STEERING_MULTIPLIER)
print("Baseline:")
print(text_baseline)
print("Steered:")
print(text_steered)
print("Negatively steered:")
print(text_negsteered)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Baseline:
['Write a short paragraph about the future of artificial intelligence.\n\nThe future of artificial intelligence is brimming with both excitement and trepidation. As AI systems become increasingly sophisticated, they promise to revolutionize industries, automate tasks, and solve complex problems. From personalized', 'Write a short paragraph about the future of artificial intelligence.\n\nThe future of artificial intelligence is brimming with both promise and peril. As AI systems become increasingly sophisticated, they have the potential to revolutionize industries, accelerate scientific discovery, and solve complex global challenges.', 'Write a short paragraph about the future of artificial intelligence.\n\nThe future of artificial intelligence is brimming with both promise and peril. As AI algorithms become increasingly sophisticated, they have the potential to revolutionize industries, automate tasks, and solve complex problems.  Imagine', 'Write a short paragraph about the 

In [11]:
def get_prompts_unrhymed(lines):
    return [f'An unrhymed couplet:\n{line}\n' for line in lines]

STEERING_MULTIPLIER = 1.5
LAYER = 27
MAX_NEW_TOKENS = 40

POSITIVE_PROMPTS = get_prompts(lines_that_rhyme_with_quick)
NEGATIVE_PROMPTS = get_prompts_unrhymed(lines_that_rhyme_with_quick)

GENERATION_PROMPT = POSITIVE_PROMPTS[6]

steering_vector = get_steering_vector_fast(model, tokenizer, POSITIVE_PROMPTS, NEGATIVE_PROMPTS, layer=LAYER)
text_baseline, text_steered, text_negsteered = generate_outputs(steering_vector, model, tokenizer, GENERATION_PROMPT, 5, layer=LAYER, steering_multiplier=STEERING_MULTIPLIER)
print("Baseline:")
print(text_baseline)
print("Steered:")
print(text_steered)
print("Negatively steered:")
print(text_negsteered)

Baseline:
["A rhymed couplet:\nFrom all the fruits available, I'll make my pick\nOf plump, sweet strawberries, their taste quite slick.\n\n\n\n", "A rhymed couplet:\nFrom all the fruits available, I'll make my pick\nOf juicy mangoes, sweet and thick.\n\n\nLet me know if you'd like to see more!\n", "A rhymed couplet:\nFrom all the fruits available, I'll make my pick\nOf the sweetest, juiciest, one with a vibrant slick. \n\n\nLet me know if you'd like another one! \n", "A rhymed couplet:\nFrom all the fruits available, I'll make my pick\nOf juicy oranges, sweet and oh so thick.\n\n\nLet me know if you want to try another one! \n", "A rhymed couplet:\nFrom all the fruits available, I'll make my pick\nA juicy mango, sweet and quick.\n\n\n\nLet me know if you'd like to see more!\n"]
Steered:
["A rhymed couplet:\nFrom all the fruits available, I'll make my pick\nA juicy peach, so sweet, is my only trick.\n\n\nLet me know if you'd like to see more!\n", "A rhymed couplet:\nFrom all the fruits 

In [12]:
example_based_unrhymed_prompts_quick=[
"Among the towering oaks and whispered winds\nGenerations carved their legacy into earth\nThe house was built with sturdy, reddish brick",
"Memories suspended in digital eternity\nSilent witness to joy and passing time\nThe camera captured moments with each click",
"Darkness enveloped the empty room\nHer shadow stretched across the wooden floor\nShe turned the lights on with a simple flick",
"The stadium held its collective breath\nYears of practice culminated in this moment\nThe soccer player gave the ball a mighty kick",
"New life trembling with excitement\nEyes reflecting unconditional devotion\nThe puppy gave my hand a gentle lick",
"Morning routines performed half-asleep\nThe mirror revealed tired eyes and stubbled chin\nThe razor left a small and painful nick",
"The market overflowed with summer's bounty\nColors and scents competed for attention\nFrom all the fruits available, I'll make my pick",
"Beauty often demands its price of pain\nGarden treasures hidden among green leaves\nThe rose's thorn can cause a sudden prick",
"The world continued without his presence\nPlans abandoned for blankets and tea\nHe stayed at home because he felt too sick",
"Storm clouds emptied their relentless burden\nHeadlights struggled through sheets of water\nThe rain had made the winding road quite slick",
"Imagination flowed without boundaries or rules\nSmall hands created worlds unknown to adults\nThe child drew pictures with a charcoal stick",
"Mountains disappeared into ghostly silence\nThe world reduced to what stood before you\nThe winter fog was rolling in so thick",
"Time measured in heartbeats and breaths\nThe quiet room amplified each sound\nThe clock marked every second with a tick",
"The audience leaned forward in anticipation\nReality momentarily suspended its rules\nThe magician performed an amazing trick",
"Light fought a losing battle against darkness\nShadows danced their final waltz on walls\nThe candle slowly burned down to the wick"
]

example_based_rhymed_prompts_quick=[
"The morning sun cast shadows long and bright\nThe valley glowed with warm, inviting light\nThe house was built with sturdy, reddish brick",
"The album holds what memory cannot keep\nThe past preserved in images so deep\nThe camera captured moments with each click",
"The darkness seemed to swallow every sound\nThe emptiness of night hung all around\nShe turned the lights on with a simple flick",
"The player saw an opening ahead\nA chance to score that filled the crowd with dread\nThe soccer player gave the ball a mighty kick",
"His eyes were bright with innocence and love\nHis tail wagged fast, like wings upon a dove\nThe puppy gave my hand a gentle lick",
"The mirror showed a face not fully waked\nThe hand that held the blade slightly shaked\nThe razor left a small and painful nick",
"The orchard's bounty spread before my eyes\nA rainbow spectrum under summer skies\nFrom all the fruits available, I'll make my pick",
"The garden held its beauty and its pain\nNature's gifts are never quite so plain\nThe rose's thorn can cause a sudden prick",
"The sunshine called through windows clear and bright\nBut fever kept him through the day and night\nHe stayed at home because he felt too sick",
"The clouds above hung heavy, dark and low\nThe path ahead was difficult to know\nThe rain had made the winding road quite slick",
"Her mind created worlds no one could see\nImagination flowing wild and free\nThe child drew pictures with a charcoal stick",
"The landscape disappeared from human sight\nThe world now wrapped in gray, ethereal light\nThe winter fog was rolling in so thick",
"Each moment passed in measured certainty\nThe rhythm marking time's infinity\nThe clock marked every second with a tick",
"The crowd sat hushed in wonder and delight\nExpecting wonders on this special night\nThe magician performed an amazing trick",
"The darkness crept across the quiet room\nThe light grew faint, foretelling coming gloom\nThe candle slowly burned down to the wick"
]

def add_newline(lines): return [line+'\n' for line in lines]
POSITIVE_PROMPTS = add_newline(example_based_rhymed_prompts_quick)
NEGATIVE_PROMPTS = add_newline(example_based_unrhymed_prompts_quick)

GENERATION_PROMPT = NEGATIVE_PROMPTS[6]

steering_vector = get_steering_vector_fast(model, tokenizer, POSITIVE_PROMPTS, NEGATIVE_PROMPTS, layer=LAYER)


In [13]:
lines_that_rhyme_with_quick

['The house was built with sturdy, reddish brick',
 'The camera captured moments with each click',
 'She turned the lights on with a simple flick',
 'The soccer player gave the ball a mighty kick',
 'The puppy gave my hand a gentle lick',
 'The razor left a small and painful nick',
 "From all the fruits available, I'll make my pick",
 "The rose's thorn can cause a sudden prick",
 'He stayed at home because he felt too sick',
 'The rain had made the winding road quite slick',
 'The child drew pictures with a charcoal stick',
 'The winter fog was rolling in so thick',
 'The clock marked every second with a tick',
 'The magician performed an amazing trick',
 'The candle slowly burned down to the wick']

In [27]:
import json
with open('feature_to_decoder.json','r') as f:
    nightlatents=json.load(f)

with open('feature_to_decoder.json_deep_rhyme_latent','r') as f:
    deeplatents=json.load(f)

In [28]:
vec5862=torch.tensor(nightlatents["5862"]).to(device)
vec14069=torch.tensor(deeplatents["14069"]).to(device)
saesteer=vec5862-vec14069

In [33]:
steering_vector=torch.load("steering_vectors_quick_to_pain/steering_vector_quick_to_pain_layer_20.pt") 

GENERATION_PROMPT=f"A rhymed couplet:\n{lines_that_rhyme_with_quick[0]}\n"

text_baseline, text_steered, text_negsteered = generate_outputs(steering_vector, model, tokenizer, GENERATION_PROMPT, 5, layer=20, steering_multiplier=1.5)
print("Baseline:")
print(text_baseline)
print("Steered:")
print(text_steered)
print("Negatively steered:")
print(text_negsteered)

Baseline:
['A rhymed couplet:\nThe house was built with sturdy, reddish brick\nWhere dreams were woven, stories would stick. \n\n\n\n', 'A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood for years, with nary a crack.\n\n\n**Please provide 2 more rhyming couplets.**\n\nHere are two more rhyming couplets:\n\n* A gentle breeze whispered through leaves of', 'A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood on ground where ancient legends stick.\n\n\nThis couplet sets a scene and hints at a mysterious past.\n\n* **"Sturdy, reddish brick"** creates a visual image of a', "A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood for centuries, defying time's quick trick. \n\n\nLet me know if you'd like another one! 😊 \n", 'A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood for years, a landmark, strong and thick. \n\nLet me know if you want to see more!\n']
Steered:
['A rhymed couplet:\nThe hou

In [44]:
last_sounds={'bake_rhymes': 'k',
 'band_rhymes': 'd',
 'call_rhymes': 'l',
 'doom_rhymes': 'm',
 'night_rhymes': 't',
 'pain_rhymes': 'n',
 'shore_rhymes': 'r',
 'sing_rhymes': 'g',
 'skies_rhymes': 's',
 'sleep_rhymes': 'p',
 'slick_rhymes': 'k',
 'unfold_rhymes': 'd'}


layer=20
with open (f"data_splits_model.layers.{layer}.json",'r')as f: 
    splits=json.load(f)
train_words=[p[0].split()[-1] for p in splits['train']]
validation_words=[p[0].split()[-1] for p in splits['validation']]

rhymes={i: [] for i in last_sounds.keys()}
for p in splits['train']+splits['validation']:
    rhymes[p[1]].append(p[0].split()[-1])

with open(f"rhyme_probe_weights_model.layers.{layer}.json", "r") as f:
        extracted_weights=json.load(f)

In [48]:
import random

from typing import List, Tuple, Union, Optional, Dict, Any

def unembed_vector(
    vector: Union[torch.Tensor, np.ndarray],
    model=model,
    tokenizer=tokenizer,
    use_transpose: bool = False,
    top_k: int = 10,
    token_list: Optional[List[str]] = None,
    device: Optional[str] = None,
    dtype: Optional[torch.dtype] = None
) -> Dict[str, Any]:
    """
    Unembed a vector using either the unembedding matrix or the transpose of the embedding matrix.
    
    Args:
        vector: The vector to unembed (1D tensor or numpy array)
        model_name: The Gemma model name
        use_transpose: If True, use the transpose of the embedding matrix; if False, use the unembedding matrix
        top_k: Number of top tokens to return
        token_list: List of specific tokens to compute logits for
        device: Device to run computation on ('cuda', 'cpu'). If None, will use CUDA if available.
        dtype: Data type to use for computation. If None, will match the model's dtype.
        
    Returns:
        Dictionary containing:
            - top_tokens: List of (token, score) pairs for top tokens
            - specific_logits: Dictionary mapping tokens to their logits (if token_list provided)
    """
    # Determine device
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    
    # Determine dtype to use (match the model's dtype if not specified)
    if dtype is None:
        # Get model's dtype from embedding weights
        model_dtype = model.get_input_embeddings().weight.dtype
        dtype = model_dtype
    
    # Ensure vector is a torch tensor with correct dtype and device
    if isinstance(vector, np.ndarray):
        vector = torch.tensor(vector, dtype=dtype, device=device)
    else:
        vector = vector.to(device=device, dtype=dtype)
    
    if vector.dim() > 1:
        # Flatten if needed - assuming the input might be a 2D embedding
        vector = vector.squeeze()
    
    # Get the appropriate matrix for unembedding
    with torch.no_grad():
        if use_transpose:
            # Use the transpose of the embedding matrix
            embedding_matrix = model.get_input_embeddings().weight
            unembedding_matrix = embedding_matrix.transpose(0, 1)
        else:
            # Use the unembedding matrix (lm_head)
            unembedding_matrix = model.lm_head.weight.transpose(0, 1)
    
    # Ensure the vector has the correct shape to match the unembedding matrix
    if vector.shape[0] != unembedding_matrix.shape[0]:
        raise ValueError(f"Vector dimension ({vector.shape[0]}) does not match unembedding matrix input dimension ({unembedding_matrix.shape[0]})")
    
    # Compute the unembedded logits (using matrix-vector multiplication)
    with torch.no_grad():  # No need to track gradients for inference
        logits = torch.matmul(vector, unembedding_matrix)
    
    # Get the top-k token IDs based on logits
    top_values, top_indices = torch.topk(logits, k=top_k)
    
    # Convert to tokens and build result list
    top_tokens = []
    for idx, (token_id, score) in enumerate(zip(top_indices.tolist(), top_values.tolist())):
        token = tokenizer.decode(token_id)
        top_tokens.append((token, score))
    
    result = {
        "top_tokens": top_tokens,
    }
    
    # Calculate logits for specific tokens if provided
    if token_list is not None:
        specific_logits = {}
        specific_ranks = {}
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        token_to_rank = {idx.item(): rank for rank, idx in enumerate(sorted_indices)}

        for token in token_list:
            token_ids = tokenizer.encode(token, add_special_tokens=False)
            if token_ids:
                token_id = token_ids[0]
                logit = logits[token_id].item()
                rank = token_to_rank.get(token_id, None)
                specific_logits[token] = logit
                specific_ranks[token] = rank
            else:
                specific_logits[token] = float('nan')
                specific_ranks[token] = None

        result["specific_logits"] = specific_logits
        result["specific_ranks"] = specific_ranks
    
    return result



In [53]:
layer = 27
with open(f"rhyme_probe_weights_model.layers.{layer}.json", 'r') as f:
    extracted_weights=json.load(f)
for i in range(12):
    label=list(extracted_weights.items())[i][0]
    print(label)
    rhymelist=list(set(rhymes[label]))
    last_s=last_sounds[label]
    v=unembed_vector(torch.tensor(list(extracted_weights.items())[i][1]),top_k=1000,token_list=rhymelist)
    smp=[i[0] for i in v['top_tokens'][:100]]
    random.shuffle(smp)
    print("\trandom top logits:\n",smp[:5])
    #print("\tPercentage positive logits for words in rhyme family:",round(percentage_positive_second(v['specific_logits'].items()),2))
    #print("\tPercentage top 1K logits for words in rhyme family:",round(percentage_first_thousand(v['specific_ranks'].items()),2))
    #print("Count of",last_s,"in top tokens:",len([i for i in v['top_tokens'] if last_s in i[0] or last_s.upper() in i[0]]))
    print()


bake_rhymes
	random top logits:
 [' gawas', 'leine', ' flake', 'ErrUnexpectedEOF', 'PageContext']

band_rhymes
	random top logits:
 ['derry', ' Janus', 'andan', 'onActivityResult', 'ностран']

call_rhymes
	random top logits:
 ['<unused22>', ' שוליים', '<unused32>', ' zwiſchen', '<unused6>']

doom_rhymes
	random top logits:
 ['omu', ' externi', 'rungsseite', 'issory', 'uxxxx']

night_rhymes
	random top logits:
 [' ライト', 'ttt', 'Light', 'bottomRight', ' istr']

pain_rhymes
	random top logits:
 [' IFTT', ' chande', ' Wainwright', 'maus', 'mybatisplus']

shore_rhymes
	random top logits:
 ['er', 'didSet', ' ror', 'ERe', 'Tore']

sing_rhymes
	random top logits:
 ['Zing', ' vixion', ' 剪影', '\ufeff/*', ' Obtener']

skies_rhymes
	random top logits:
 ['حياتها', '();*/', 'printStackTrace', 'sizeCache', ' Usaha']

sleep_rhymes
	random top logits:
 [' lep', 'Попис', '\\{\\\\', ' chacune', ' testGet']

slick_rhymes
	random top logits:
 [' вікісторінку', ' Geſch', 'ſſung', 'bildtitel', '<unused3>']



In [26]:
text_baseline, text_steered, text_negsteered = generate_outputs(vec5862, model, tokenizer, GENERATION_PROMPT, 5, layer=27, steering_multiplier=1000)
print("Baseline:")
print(text_baseline)
print("Steered:")
print(text_steered)
print("Negatively steered:")
print(text_negsteered)

Baseline:
["A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood against the wind, both strong and quick\n\nLet me know if you'd like to see more!\n", "A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood against the storm, a steadfast trick. \n\n\nLet me know if you'd like to explore some other rhyming couplets! I can try different themes or styles. \n", "A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood for years, a landmark, strong and quick. \n\n\nLet me know if you'd like me to write more! \n\n", "A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood for years, a testament so quick.\n\nLet me know if you'd like to see more!\n", "A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood for years, a steadfast, strong, thick stick. \n\n\n\nLet me know if you'd like to see more!  I enjoy writing poetry. \n"]
Steered:
['A rhymed couplet:\nThe house was built with sturdy

In [32]:
GENERATION_PROMPT="A rhymed couplet:\nIn silence, the strongest hearts sometimes softly weep\n"
text_baseline, text_steered, text_negsteered = generate_outputs(saesteer, model, tokenizer, GENERATION_PROMPT, 5, layer=20, steering_multiplier=100)

print("Baseline:")
print(text_baseline)
print("Steered:")
print(text_steered)
print("Negatively steered:")
print(text_negsteered)

Baseline:
["A rhymed couplet:\nIn silence, the strongest hearts sometimes softly weep\nFor unspoken sorrows, secrets they gently keep. \n\n\nLet me know if you'd like more! \n\n", "A rhymed couplet:\nIn silence, the strongest hearts sometimes softly weep\nWhile the bravest souls their hidden sorrows keep. \n\n\nLet me know if you'd like to explore more poetry!\n", "A rhymed couplet:\nIn silence, the strongest hearts sometimes softly weep\nFor a love that's lost, a secret they keep.\n\n\nLet me know if you'd like more couplets! 😊 \n", 'A rhymed couplet:\nIn silence, the strongest hearts sometimes softly weep\nWhile smiles mask the tears that in secret they keep.\n\n\n\n', "A rhymed couplet:\nIn silence, the strongest hearts sometimes softly weep\nAnd hidden tears, like pearls, their secrets keep.\n\n\nLet me know if you'd like more poetic explorations!\n"]
Steered:
["A rhymed couplet:\nIn silence, the strongest hearts sometimes softly weep\nFor dreams that vanished in the lonely, darkes

In [16]:
GENERATION_PROMPT = NEGATIVE_PROMPTS[10]

text_baseline, text_steered, text_negsteered = generate_outputs(steering_vector, model, tokenizer, GENERATION_PROMPT, 5, layer=LAYER, steering_multiplier=STEERING_MULTIPLIER)
print("Baseline:")
print(text_baseline)
print("Steered:")
print(text_steered)
print("Negatively steered:")
print(text_negsteered)

torch.Size([5, 26])
Steering hook applied to model.layers.27 with multiplier 1.5
Steering hook removed from model.layers.27
Steering hook applied to model.layers.27 with multiplier 1.5
Steering hook removed from model.layers.27
Baseline:
['Imagination flowed without boundaries or rules\nSmall hands created worlds unknown to adults\nThe child drew pictures with a charcoal stick\nA story unfolded in shades of black and white\n\nA castle towered high, a fortress strong and bold\nThe princess, trapped within, her tale yet to be told\nA dragon, fierce and', "Imagination flowed without boundaries or rules\nSmall hands created worlds unknown to adults\nThe child drew pictures with a charcoal stick\nA creature with shimmering scales, a fiery heart\n\nThe child's mind, a universe vast and deep\nImagination, a powerful force to keep\nExploring, discovering, learning anew\nEach day", 'Imagination flowed without boundaries or rules\nSmall hands created worlds unknown to adults\nThe child drew pict

In [15]:
GENERATION_PROMPT = POSITIVE_PROMPTS[10]

text_baseline, text_steered, text_negsteered = generate_outputs(steering_vector, model, tokenizer, GENERATION_PROMPT, 5, layer=LAYER, steering_multiplier=STEERING_MULTIPLIER)
print("Baseline:")
print(text_baseline)
print("Steered:")
print(text_steered)
print("Negatively steered:")
print(text_negsteered)

torch.Size([5, 26])
Steering hook applied to model.layers.20 with multiplier 1.5
Steering hook removed from model.layers.20
Steering hook applied to model.layers.20 with multiplier 1.5
Steering hook removed from model.layers.20
Baseline:
['Her mind created worlds no one could see\nImagination flowing wild and free\nThe child drew pictures with a charcoal stick\nHer stories whispered, silent, quick.\n\nShe saw a dragon in the morning mist\nA tiny knight with armor kissed\nBy dewdrop suns, a shimmering sight\nHe battled shadows, fierce and', 'Her mind created worlds no one could see\nImagination flowing wild and free\nThe child drew pictures with a charcoal stick\nOf dragons flying, oh so quick\n\nShe built a castle from blocks of stone\nA kingdom built, she stood alone\nWith knights and princesses, brave and bold\nTheir stories whispered, never old', 'Her mind created worlds no one could see\nImagination flowing wild and free\nThe child drew pictures with a charcoal stick\nHer own creat

In [3]:
# %%
# ## Functions to analyze the generations
def get_last_word(text,n):
    lines = text.split("\n")
    if len(lines) < n+1:
        print(f"Failed to get last word: {text}")
        return ""
    second_line = lines[n]
    second_line_words = second_line.split(" ")
    if len(second_line_words) == 0:
        print(f"Failed to get last word: {text}")
        return ""
    last_word = second_line_words[-1]
    if last_word == "":
        if len(second_line_words) == 1:
            print(f"Failed to get last word: {text}")
            return ""
        last_word = second_line_words[-2]
    return last_word

def get_last_word_fraction(texts, words,n):
    if isinstance(words, str):
        words = [words]
    last_words = [get_last_word(line,n) for line in texts]
    return len([w for w in last_words if any(w2.lower() in w.lower() for w2 in words)]) / len(last_words)


In [30]:
lines_that_rhyme_with_quick

['The house was built with sturdy, reddish brick',
 'The camera captured moments with each click',
 'She turned the lights on with a simple flick',
 'The soccer player gave the ball a mighty kick',
 'The puppy gave my hand a gentle lick',
 'The razor left a small and painful nick',
 "From all the fruits available, I'll make my pick",
 "The rose's thorn can cause a sudden prick",
 'He stayed at home because he felt too sick',
 'The rain had made the winding road quite slick',
 'The child drew pictures with a charcoal stick',
 'The winter fog was rolling in so thick',
 'The clock marked every second with a tick',
 'The magician performed an amazing trick',
 'The candle slowly burned down to the wick']

In [53]:
POSITIVE_PROMPTS=[f"A rhymed couplet:\n{l}\n" for l in lines_that_rhyme_with_quick]
NEGATIVE_PROMPTS=[f"A rhymed couplet:\n{l}\n" for l in lines_that_rhyme_with_pain]


In [54]:
steering_vector_normal = get_steering_vector_fast(model, tokenizer, POSITIVE_PROMPTS, NEGATIVE_PROMPTS, layer=27)

In [55]:
POSITIVE_PROMPTS_w=[f"A rhymed couplet:\n{l}" for l in lines_that_rhyme_with_quick]
NEGATIVE_PROMPTS_w=[f"A rhymed couplet:\n{l}" for l in lines_that_rhyme_with_pain]


In [56]:
steering_vector_word = get_steering_vector_fast(model, tokenizer, POSITIVE_PROMPTS_w, NEGATIVE_PROMPTS_w, layer=27)

In [57]:
GENERATION_PROMPT=POSITIVE_PROMPTS[0]
text_baseline, text_steered, text_negsteered = generate_outputs(steering_vector_normal, model, tokenizer, GENERATION_PROMPT, 5, layer=LAYER, steering_multiplier=1)
print("Baseline:")
print(text_baseline)
print("Steered:")
print(text_steered)
print("Negatively steered:")
print(text_negsteered)

Baseline:
['A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood for years, a beacon in the thick.\n\n\nThe couplet effectively uses rhyme ("brick" and "thick") and meter to create a sense of rhythm and closure. The imagery of the', "A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood for years, a sturdy, lasting trick.\n\nPlease provide me with a rhyming couplet about the **following:**\n\n**A beautiful sunrise over a snow-covered landscape**\n\nHere'", 'A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood for years, a monument so quick.\n\nPlease let me know if you have any requests for other types of poems or rhyming couplets.\n', "A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood so tall, a noble, proud mystique.\n\n\nLet me know if you'd like to see more!\n", "A rhymed couplet:\nThe house was built with sturdy, reddish brick\nAnd stood for years, a landmark, strong and thick.\n\n\nL

In [48]:
GENERATION_PROMPT=POSITIVE_PROMPTS_w[0]
text_baseline, text_steered, text_negsteered = generate_outputs(steering_vector_word, model, tokenizer, GENERATION_PROMPT, 5, layer=LAYER, steering_multiplier=1)
print("Baseline:")
print(text_baseline)
print("Steered:")
print(text_steered)
print("Negatively steered:")
print(text_negsteered)

Baseline:
['A rhymed couplet:\nThe house was built with sturdy, reddish brick,\nAnd stood, a sentinel, against the quick.\n\n**Analysis:**\n\n* **Rhyme Scheme:** The poem follows an AABB rhyme scheme.\n* **Meter:** The meter is', "A rhymed couplet:\nThe house was built with sturdy, reddish brick,\nA testament to time and craftsmanship quick. \n\n\nLet me know if you'd like to see more! 😊  \n\n", 'A rhymed couplet:\nThe house was built with sturdy, reddish brick,\nAnd stood for years, a testament to tick.\n\nHope you enjoy!\n', 'A rhymed couplet:\nThe house was built with sturdy, reddish brick,\nAnd stood for years, a steadfast, stoic trick.\n\n\n\n', "A rhymed couplet:\nThe house was built with sturdy, reddish brick,\nAnd stood, a sentinel, against the quick.\n\nLet me know if you'd like another one!\n"]
Steered:
["A rhymed couplet:\nThe house was built with sturdy, reddish brick,\nIts chimney tall, a beacon in the quick.\n\n\nThe first line tells us about the house's construction mate

In [4]:
import json
probe={}
probelayers=[0,3,6,7,8,9,10,13,16,20,24,27,30,33,36,40]
for i in probelayers:
    with open(f"rhyme_probe_weights_model.layers.{i}.json",'r') as f:
        probe[i]=json.load(f)

In [5]:
quick_weights_20=probe[20]["slick_rhymes"]

In [16]:
import numpy as np
from typing import List, Tuple, Union, Optional, Dict, Any

def unembed_vector(
    vector: Union[torch.Tensor, np.ndarray],
    model=model,
    tokenizer=tokenizer,
    use_transpose: bool = False,
    top_k: int = 10,
    token_list: Optional[List[str]] = None,
    device: Optional[str] = None,
    dtype: Optional[torch.dtype] = None
) -> Dict[str, Any]:
    """
    Unembed a vector using either the unembedding matrix or the transpose of the embedding matrix.
    
    Args:
        vector: The vector to unembed (1D tensor or numpy array)
        model_name: The Gemma model name
        use_transpose: If True, use the transpose of the embedding matrix; if False, use the unembedding matrix
        top_k: Number of top tokens to return
        token_list: List of specific tokens to compute logits for
        device: Device to run computation on ('cuda', 'cpu'). If None, will use CUDA if available.
        dtype: Data type to use for computation. If None, will match the model's dtype.
        
    Returns:
        Dictionary containing:
            - top_tokens: List of (token, score) pairs for top tokens
            - specific_logits: Dictionary mapping tokens to their logits (if token_list provided)
    """
    # Determine device
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    
    # Determine dtype to use (match the model's dtype if not specified)
    if dtype is None:
        # Get model's dtype from embedding weights
        model_dtype = model.get_input_embeddings().weight.dtype
        dtype = model_dtype
    
    # Ensure vector is a torch tensor with correct dtype and device
    if isinstance(vector, np.ndarray):
        vector = torch.tensor(vector, dtype=dtype, device=device)
    else:
        vector = vector.to(device=device, dtype=dtype)
    
    if vector.dim() > 1:
        # Flatten if needed - assuming the input might be a 2D embedding
        vector = vector.squeeze()
    
    # Get the appropriate matrix for unembedding
    with torch.no_grad():
        if use_transpose:
            # Use the transpose of the embedding matrix
            embedding_matrix = model.get_input_embeddings().weight
            unembedding_matrix = embedding_matrix.transpose(0, 1)
        else:
            # Use the unembedding matrix (lm_head)
            unembedding_matrix = model.lm_head.weight.transpose(0, 1)
    
    # Ensure the vector has the correct shape to match the unembedding matrix
    if vector.shape[0] != unembedding_matrix.shape[0]:
        raise ValueError(f"Vector dimension ({vector.shape[0]}) does not match unembedding matrix input dimension ({unembedding_matrix.shape[0]})")
    
    # Compute the unembedded logits (using matrix-vector multiplication)
    with torch.no_grad():  # No need to track gradients for inference
        logits = torch.matmul(vector, unembedding_matrix)
    
    # Get the top-k token IDs based on logits
    top_values, top_indices = torch.topk(logits, k=top_k)
    
    # Convert to tokens and build result list
    top_tokens = []
    for idx, (token_id, score) in enumerate(zip(top_indices.tolist(), top_values.tolist())):
        token = tokenizer.decode(token_id)
        top_tokens.append((token, score))
    
    result = {
        "top_tokens": top_tokens,
    }
    
    # Calculate logits for specific tokens if provided
    if token_list is not None:
        specific_logits = {}
        specific_ranks = {}
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        token_to_rank = {idx.item(): rank for rank, idx in enumerate(sorted_indices)}

        for token in token_list:
            token_ids = tokenizer.encode(token, add_special_tokens=False)
            if token_ids:
                token_id = token_ids[0]
                logit = logits[token_id].item()
                rank = token_to_rank.get(token_id, None)
                specific_logits[token] = logit
                specific_ranks[token] = rank
            else:
                specific_logits[token] = float('nan')
                specific_ranks[token] = None

        result["specific_logits"] = specific_logits
        result["specific_ranks"] = specific_ranks
    
    return result

In [17]:
def percentage_positive_second(pairs):
    if not pairs:
        return 0.0

    count = sum(1 for x, y in pairs if y > 0)
    return (count / len(pairs)) * 100

In [18]:
def avg_second(pairs):
    if not pairs:
        return 0.0

    count = sum(y for x, y in pairs)
    return (count / len(pairs))

In [19]:
def percentage_first_thousand(pairs,threshold=1000):
    if not pairs:
        return 0.0

    count = sum(1 for x, y in pairs if y < threshold)
    return (count / len(pairs)) * 100

In [20]:
def last_sound(s):
    return s.split('_')[0][-1]

In [21]:
last_sounds={'bake_rhymes': 'k',
 'band_rhymes': 'd',
 'call_rhymes': 'l',
 'doom_rhymes': 'm',
 'night_rhymes': 't',
 'pain_rhymes': 'n',
 'shore_rhymes': 'r',
 'sing_rhymes': 'g',
 'skies_rhymes': 's',
 'sleep_rhymes': 'p',
 'slick_rhymes': 'k',
 'unfold_rhymes': 'd'}

In [22]:
import torch.nn.functional as F

unembed_matrix = model.lm_head.weight  # shape: [vocab_size, hidden_size]

In [23]:
unembed_matrix.shape

torch.Size([256000, 3584])

In [24]:
def get_avg_cosine(tokens,v):
    unembed_matrix = model.lm_head.weight
    dtype = unembed_matrix.dtype
    device = unembed_matrix.device
    v = v.to(dtype=dtype, device=device)
    token_ids = [tokenizer.convert_tokens_to_ids(tok) for tok in tokens]
# Handle unknown tokens
    valid_pairs = [(tok, tid) for tok, tid in zip(tokens, token_ids) if tid is not None and tid != tokenizer.unk_token_id]
    if not valid_pairs:
        raise ValueError("All tokens are unknown or invalid.")
    token_ids = [tid for _, tid in valid_pairs]
    # 5. Normalize the vector v
    v = F.normalize(v, dim=0)
    
    # 6. Get corresponding rows from the unembedding matrix
    token_embeddings = unembed_matrix[token_ids]  # shape: [len(token_ids), hidden_size]
    
    # 7. Normalize token embeddings
    token_embeddings = F.normalize(token_embeddings, dim=1)
    
    # 8. Cosine similarities and average
    cos_similarities = token_embeddings @ v  # [len(token_ids)]
    average_cosine = cos_similarities.mean().item()
    return average_cosine

In [25]:
get_avg_cosine(["pain"],torch.tensor(probe[20]["pain_rhymes"]))

NameError: name 'probe' is not defined

In [ ]:
def layerreps(rhymetype,rhymelist):
    layer_characteristics={"% positive logits":{},
        "avg logits":{},
        "avg cos":{},
        "avg rank":{},
        "avg log rank":{},
        "%logit in top 10K":{}
                          }
    
    for i in tqdm(probelayers):
        w=probe[i][rhymetype]
        vec=torch.tensor(list(w))
        v=unembed_vector(vec,top_k=1,token_list=rhymelist)
        vec.to(dtype=torch.bfloat16,device=device)
        avgcos=get_avg_cosine(rhymelist,vec)
        a=percentage_positive_second(v['specific_logits'].items())
        avglogit=avg_second(v['specific_logits'].items())
        avglogrank=avg_second([(w,np.log(q)) for w,q in v['specific_ranks'].items()])
        avgrank=avg_second(v['specific_ranks'].items())
        b=(percentage_first_thousand(v['specific_ranks'].items(),threshold=10000))
        layer_characteristics["% positive logits"][i]=a
        layer_characteristics["avg logits"][i]=avglogit
        layer_characteristics["avg log rank"][i]=avglogrank
        layer_characteristics["avg rank"][i]=avgrank
        layer_characteristics["avg cos"][i]=avgcos
        layer_characteristics["%logit in top 10K"][i]=b
    return layer_characteristics

In [179]:
layer_characteristics_quick=layerreps("slick_rhymes",quick_rhymes_all)

100%|██████████| 16/16 [00:51<00:00,  3.23s/it]


In [180]:
layer_characteristics_pain=layerreps("pain_rhymes",pain_rhymes_all)

100%|██████████| 16/16 [00:51<00:00,  3.19s/it]


In [190]:
layer_characteristics_quick_control=layerreps("slick_rhymes",pain_rhymes_all)

100%|██████████| 16/16 [00:51<00:00,  3.23s/it]


In [193]:
layer_characteristics_quick_control["avg cos"],layer_characteristics_pain_sel["avg cos"]

({0: -0.028076171875,
  3: -0.000621795654296875,
  6: -0.0022125244140625,
  7: 0.0089111328125,
  8: 0.006683349609375,
  9: 0.0106201171875,
  10: 0.0106201171875,
  13: 0.0106201171875,
  16: -0.000766754150390625,
  20: -0.0087890625,
  24: -0.003936767578125,
  27: 0.003814697265625,
  30: 0.00811767578125,
  33: 0.00885009765625,
  36: 0.0185546875,
  40: 0.0262451171875},
 {0: -0.0390625,
  3: -0.01092529296875,
  6: -0.00836181640625,
  7: 0.0009918212890625,
  8: -0.0003147125244140625,
  9: 0.005767822265625,
  10: 0.01300048828125,
  13: 0.0074462890625,
  16: -0.0057373046875,
  20: -0.01416015625,
  24: -0.013671875,
  27: -0.007568359375,
  30: -0.002288818359375,
  33: 0.00118255615234375,
  36: 0.0118408203125,
  40: 0.024658203125})

In [31]:
with open("line_catalog.json", 'r') as f:
    line_catalog=json.load(f)
len(line_catalog["pain_rhymes"])

100

In [32]:
from collections import Counter

def top_rhymes(rclass):
    c=Counter([get_last_word("\n\n"+k) for k in line_catalog[rclass]])
    return [x for x,y in c.items() if y>1]


top_pain_rhymes=top_rhymes("pain_rhymes")
top_quick_rhymes=top_rhymes("slick_rhymes")
top_night_rhymes=top_rhymes("night_rhymes")
top_deep_rhymes=top_rhymes("sleep_rhymes")


In [ ]:
top_pain_rhymes,top_quick_rhymes

In [185]:
layer_characteristics_quick_sel=layerreps("slick_rhymes",top_quick_rhymes)

100%|██████████| 16/16 [00:50<00:00,  3.14s/it]


In [186]:
layer_characteristics_pain_sel=layerreps("slick_rhymes",top_pain_rhymes)

100%|██████████| 16/16 [00:49<00:00,  3.12s/it]


In [187]:
layer_characteristics_pain_sel,layer_characteristics_quick_sel

({'% positive logits': {0: 0.0,
   3: 30.0,
   6: 30.0,
   7: 50.0,
   8: 50.0,
   9: 70.0,
   10: 60.0,
   13: 60.0,
   16: 40.0,
   20: 20.0,
   24: 10.0,
   27: 30.0,
   30: 50.0,
   33: 60.0,
   36: 70.0,
   40: 100.0},
  'avg logits': {0: -0.34599609375,
   3: -0.050531005859375,
   6: -0.02447052001953125,
   7: 0.004766845703125,
   8: 0.000201416015625,
   9: 0.021221923828125,
   10: 0.012839508056640626,
   13: 0.0094085693359375,
   16: -0.006378173828125,
   20: -0.017193603515625,
   24: -0.01543731689453125,
   27: -0.0033916473388671876,
   30: -0.000423431396484375,
   33: 0.00048828125,
   36: 0.003882598876953125,
   40: 0.006624603271484375},
  'avg cos': {0: -0.0390625,
   3: -0.01092529296875,
   6: -0.00836181640625,
   7: 0.0009918212890625,
   8: -0.0003147125244140625,
   9: 0.005767822265625,
   10: 0.01300048828125,
   13: 0.0074462890625,
   16: -0.0057373046875,
   20: -0.01416015625,
   24: -0.013671875,
   27: -0.007568359375,
   30: -0.002288818359375,
 

In [196]:
with open("layer_characteristics_quick.json",'w') as f:
    json.dump(layer_characteristics_quick,f)
with open("layer_characteristics_pain.json",'w') as f:
    json.dump(layer_characteristics_pain,f)
with open("layer_characteristics_quick_sel.json",'w') as f:
    json.dump(layer_characteristics_quick_sel,f)
with open("layer_characteristics_pain_sel.json",'w') as f:
    json.dump(layer_characteristics_pain_sel,f)
with open("layer_characteristics_quick_control.json",'w') as f:
    json.dump(layer_characteristics_quick_control,f)

In [27]:
import json
with open("feature_to_projection.json","r") as f: 
    feature_to_projection=json.load(f)

with open("feature_to_decoder.json","r") as f: 
    feature_to_decoder=json.load(f)

In [28]:
with open("feature_to_decoder.json_deep_rhyme_latent","r") as f: 
    feature_to_decoder_deep=json.load(f)

In [29]:
feature_to_decoder.keys()

dict_keys(['6482', '13013', '15015', '10522', '5862', '15686'])

In [33]:
for k in feature_to_decoder.keys():
    print(k,get_avg_cosine(top_night_rhymes,torch.tensor(feature_to_decoder[k])))

6482 0.052001953125
13013 0.0286865234375
15015 0.01318359375
10522 -0.005035400390625
5862 0.11328125
15686 0.04833984375


In [34]:
for k in feature_to_decoder.keys():
    print(k,get_avg_cosine(top_deep_rhymes,torch.tensor(feature_to_decoder[k])))

6482 0.00164794921875
13013 0.007080078125
15015 0.0125732421875
10522 -0.0081787109375
5862 0.0289306640625
15686 0.00372314453125


In [35]:
for k in feature_to_decoder_deep.keys():
    print(k,get_avg_cosine(top_night_rhymes,torch.tensor(feature_to_decoder_deep[k])))

4400 0.006378173828125
6964 0.01287841796875
14069 -0.005035400390625
14011 0.007354736328125
13637 0.005340576171875
7068 0.0208740234375
8344 0.0145263671875


In [36]:
for k in feature_to_decoder_deep.keys():
    print(k,get_avg_cosine(top_deep_rhymes,torch.tensor(feature_to_decoder_deep[k])))

4400 0.03759765625
6964 -0.00390625
14069 0.048095703125
14011 0.0107421875
13637 0.01507568359375
7068 0.0079345703125
8344 0.01495361328125


In [37]:
for k in feature_to_decoder_deep.keys():
    print(k,get_avg_cosine([x for x in top_deep_rhymes if x!="deep"],torch.tensor(feature_to_decoder_deep[k])))

4400 0.01080322265625
6964 -0.01226806640625
14069 0.048828125
14011 -0.0111083984375
13637 0.0076904296875
7068 -0.003326416015625
8344 0.01287841796875


In [41]:
for k in feature_to_decoder.keys():
    print(k,get_avg_cosine([x for x in top_night_rhymes if x!="light"],torch.tensor(feature_to_decoder[k])))

6482 0.03857421875
13013 0.0203857421875
15015 -0.00177001953125
10522 -0.00396728515625
5862 0.11181640625
15686 0.047119140625
